## CSV Data to MySQL for use in VISDOM

This notebook can be used to construct an 'account' table and a 'meter_data' table in mysql, based on the csv files with data extracted from the prop39schools xml files.

### Directory location

In the case below, it is reading from the sample csv folder. Generally just point this to the directory with all of the csv files.

In [1]:
csv_dir = "../csv"

### Notebook config

In [2]:
import pandas as pd
import numpy as np
import mysql.connector, os
from sqlalchemy import create_engine

In [3]:
engine = create_engine('mysql+mysqlconnector://root@localhost/visdom_data_sample', echo=False)
conn = engine.connect()

### Reading the account table data from the _BILL.csv files

In [4]:
usecols = [
    'utility',
    'customer_name',
    'customer_city',
    'customer_zip',
    'customer_account',
    'lea_customer',
    'cds_code',
    'school_site_name',
    'school_city',
    'school_site_zip',
    'agreement',
    'rate_schedule_id'    
]

In [5]:
accounts_list = []
for root, dirs, files in os.walk(csv_dir):
    for f in files:
        if f.endswith('_BILL.csv'):
            df = pd.read_csv(os.path.join(root,f), usecols=usecols)
            df = df.drop_duplicates()
            accounts_list.extend(df.to_dict(orient='records'))

accounts_df = df.from_records(accounts_list)
accounts_df = accounts_df.drop_duplicates()
print len(accounts_df)
accounts_df.head(3)

8


,agreement,cds_code,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility
0,8496493494,1100170130419,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Community,94544-1136,PacificGasElectric
1,5637199045,1612346001481,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,A10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric
2,5637199687,1612346001481,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,HA10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric


In [6]:
accounts_df.columns.tolist()

['agreement',
 'cds_code',
 'customer_account',
 'customer_city',
 'customer_name',
 'customer_zip',
 'lea_customer',
 'rate_schedule_id',
 'school_city',
 'school_site_name',
 'school_site_zip',
 'utility']

In [7]:
accounts_df.columns = [
 'meter_uuid',
 'account_uuid',
 'customer_account',
 'customer_city',
 'customer_name',
 'customer_zip',
 'lea_customer',
 'rate_schedule_id',
 'school_city',
 'school_site_name',
 'school_site_zip',
 'utility_name'
]

In [8]:
accounts_df['zip5'] = accounts_df['school_site_zip'].str[:5]
accounts_df.head(3)

,meter_uuid,account_uuid,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility_name,zip5
0,8496493494,1100170130419,NaN,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,NaN,E19S,Hayward,Alameda County Community,94544-1136,PacificGasElectric,94544
1,5637199045,1612346001481,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,A10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric,94560
2,5637199687,1612346001481,NaN,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,NaN,HA10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric,94560


In [9]:
reals = accounts_df[['account_uuid']].applymap(np.isreal)
accounts_df = accounts_df[reals['account_uuid']]
len(accounts_df)

8

In [10]:
accounts_df = accounts_df.drop_duplicates(subset=['meter_uuid'], keep='last')
len(accounts_df)

8

### Creating the account table in the desired format and writing to it

In [11]:
create_table_sql = '''
CREATE TABLE `account` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `account_uuid` bigint(20) DEFAULT NULL,
  `meter_uuid` bigint(20) DEFAULT NULL,
  `zip5` varchar(5) DEFAULT NULL,
  `customer_account` varchar(50) DEFAULT NULL,
  `customer_city` varchar(50) DEFAULT NULL,
  `customer_name` varchar(50) DEFAULT NULL,
  `customer_zip` varchar(10) DEFAULT NULL,
  `lea_customer` varchar(50) DEFAULT NULL,
  `rate_schedule_id` varchar(50) DEFAULT NULL,
  `school_city` varchar(50) DEFAULT NULL,
  `school_site_name` varchar(100) DEFAULT NULL,
  `school_site_zip` varchar(10) DEFAULT NULL,
  `utility_name` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`id`),
  KEY `zip5_meter_uuid_idx` (`METER_UUID`,`zip5`),
  KEY `account_uuid_idx` (`ACCOUNT_UUID`),
  KEY `meter_uuid_idx` (`METER_UUID`)
)
'''

conn.execute('DROP TABLE IF EXISTS `account`;')
conn.execute(create_table_sql)

In [12]:
accounts_df.to_sql(name='account', con=engine, if_exists='append', index=False)

Quick test to make sure it's working:

In [13]:
pd.read_sql('SELECT * FROM account LIMIT 3;', con=engine)

,id,account_uuid,meter_uuid,zip5,customer_account,customer_city,customer_name,customer_zip,lea_customer,rate_schedule_id,school_city,school_site_name,school_site_zip,utility_name
0,1,1100170130419,8496493494,94544,None,HAYWARD,ALAMEDA COUNTY OFFICE OF EDUCATION,94544-1136,None,E19S,Hayward,Alameda County Community,94544-1136,PacificGasElectric
1,2,1612346001481,5637199045,94560,None,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,None,A10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric
2,3,1612346001481,5637199687,94560,None,NEWARK,NEWARK UNIFIED SCHOOL DISTRICT,94560-2554,None,HA10S,Newark,James A. Graham Elementary,94560-2318,PacificGasElectric


In [14]:
pd.read_sql('SELECT COUNT(*) FROM account;', con=engine)

,COUNT(*)
0,8


In [15]:
pd.read_sql('SELECT COUNT(DISTINCT(meter_uuid)) FROM account;', con=engine)

,COUNT(DISTINCT(meter_uuid))
0,8


### Creating the meter_data table in the desired format

Note: this is currently treating the meter_uuid, account_uuid, date and zip5 as integers, but they should more likely be treated as varchar, varchar, datetime and varchar, respectively.

In [16]:
create_table_sql = '''
CREATE TABLE `meter_data` (
  `meter_uuid` bigint(20) NOT NULL,
  `account_uuid` bigint(20) NOT NULL,
  `date` int(11) NOT NULL,
  `zip5` int(11) DEFAULT NULL,
'''

for i in range(1,97):
    create_table_sql += "`h" + str(i) + "` int(11) DEFAULT NULL,\n"

create_table_sql += '''    
  PRIMARY KEY (`meter_uuid`,`date`),
  KEY `meter_uuid_idx` (`meter_uuid`),
  KEY `account_uuid_idx` (`account_uuid`),
  KEY `zip_Date_idx` (`date`,`zip5`),
  KEY `zip_idx` (`zip5`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;
'''

conn.execute('DROP TABLE IF EXISTS `meter_data`;')
conn.execute(create_table_sql)

### Fill the table with data from the csv files

In [17]:
usecols = ['agreement', 'start']
for i in range(1,97):
    usecols.append('d' + str(i))

In [18]:
colnames = ['meter_uuid', 'date']
for i in range(1,97):
    colnames.append('h' + str(i))

In [19]:
for root, dirs, files in os.walk(csv_dir):
    for f in files:
        if f.endswith('_INTERVAL.csv'):
            df = pd.read_csv(os.path.join(root,f), usecols=usecols)
            if len(df) > 0:
                df.columns = colnames
                df = df.drop_duplicates()
                df = pd.merge(df,accounts_df[['meter_uuid','account_uuid','zip5']],on='meter_uuid')
                try:
                    df.to_sql(name='meter_data', con=engine, if_exists='append', index=False)
                except:
                    print "failed sql insert. meter_uuid:" + str(df['meter_uuid'][0]) + ", filename: " + f

### Quick tests to make sure it's done so properly

In [20]:
pd.read_sql('SELECT * FROM meter_data LIMIT 3;', con=engine)

,meter_uuid,account_uuid,date,zip5,h1,h2,h3,h4,h5,h6,...,h87,h88,h89,h90,h91,h92,h93,h94,h95,h96
0,314270951,1612006001341,1372662000,94550,7,8,6,7,6,7,...,9,10,8,9,9,9,9,8,8,9
1,314270951,1612006001341,1372748400,94550,8,9,8,8,8,8,...,9,9,8,9,8,9,8,9,9,8
2,314270951,1612006001341,1372834800,94550,9,9,9,8,9,8,...,9,9,9,9,9,9,9,9,9,9


In [21]:
pd.read_sql('SELECT COUNT(*) FROM meter_data;', con=engine)

,COUNT(*)
0,2167
